In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Mô tả bài toán
> ***Bài toán đặt ra là với các câu hỏi trên Quora có phải là câu hỏi toxic hay không (Question Classification) ?***
* Input: Các câu hỏi dưới dạng text.
* Output: Yes or No ?


# Công việc cần làm
**1. Xử lý dữ liệu**
> Dữ liệu đầu vào là text nên chúng ta cần xử lý trước khi train
* loại bỏ dấu câu, số, stop_words, .....
* xử lý lại nghĩa của từ

**2. Training dữ liệu với mô hình Logistic Regression**
> Đối với bài toán này, ban đầu mình sử dụng mô hình **Logistic Regression** 
>  Mô hình này giống với **Linear Regression** ở khía cạnh đầu ra là số thực, và giống với **PLA** ở việc đầu ra bị chặn (trong đoạn 0 -> 1). Mặc dù trong tên có chứa từ "regression", tuy vậy **Logistic Regression** thường được sử dụng nhiều cho các bài toán **classification**. Do vậy, ban đầu mình lựa chọn **Logistic Regression** cho bài toán này. Sau đó mình sẽ lựa chọn một mô hình khác để mong đạt được kết quả cao hơn.

**3. Đánh giá mô hình Training LR, sử dụng mô hình khác để đạt kết quả cao hơn**

**4. Xử dụng mô hình tốt nhất để giải quyết dữ liệu Test**

In [ ]:
import pandas as pd 
import seaborn as sns

# 1. Xử lý dữ liệu

**Đọc dữ liệu vào từ định dạng csv**

**Dữ liệu vào gồm 3 loại**

* Sample submission
* Train
* Test


In [ ]:
df_sample_sub = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv')
df_train = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
df_test = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

**Mình xem thử từng loại dữ liệu sẽ như thế nào**

In [ ]:
df_sample_sub.head(5)

In [ ]:
df_train.head(5)

**Dữ liệu vào gồm các Question_text là ngôn ngữ tự nhiên. Nếu để nguyên, có vẻ vô cùng khó để train. Vì vậy chúng ta cần xử lý các Question_text.**
> Dấu câu, số, các "stopword", và nhiều từ đồng nghĩa chúng ta cần xử lý lại.

In [ ]:
df_train.info()

In [ ]:
df_train.target.value_counts()

**Đối với tập dữ liệu Train**

**Có 1225312 câu hỏi bình thường và 80810 câu hỏi Toxic
Có thể thấy là khá là nhiều câu hỏi Toxic.**

In [ ]:
sns.countplot(data=df_train, x='target')

**Để xử lý dữ liệu vào, mình xử dụng Natural Language Toolkit**

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk_stopwords = stopwords.words('english')

wordnet_lemmatizer = WordNetLemmatizer()

def lemSentence(sentence):
    token_words = word_tokenize(sentence)
    lem_sentence = []
    for word in token_words:
        lem_sentence.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        lem_sentence.append(" ")
    return "".join(lem_sentence)

In [ ]:
def clean(message, lem = True):
    # Loại bỏ dấu câu
    message = message.translate(str.maketrans('', '', string.punctuation))
    
    # Loại bỏ số
    message = message.translate(str.maketrans('', '', string.digits))
    
    # Loại bỏ "stopwords"
    message = [word for word in word_tokenize(message) if not word.lower() in nltk_stopwords]
    message = ' '.join(message)
    
    # Xử lý lại nghĩa của từ
    if lem:
        message = lemSentence(message)
    
    return message

In [ ]:
# Clean các câu hỏi
df_train['question_text_cleaned'] = df_train.question_text.apply(lambda x: clean(x, True))

**Có thể thấy dữ liệu sau khi xử lý đã "sạch" hơn so với ban đầu như bảng dưới**

In [ ]:
df_train.head(5)

# 2. Training dữ liệu với mô hình Logistic Regression
> **Như đã trình bày ở trên, mình sử dụng LR để train**

> ***Nói qua về Logistic Regression***
>> Có lẽ các bạn đều đã biết về hai mô hình tuyến tính (linear models) **Linear Regression** và **Perceptron Learning Algorithm (PLA)**. Trong **Linear Regression**, người ta sử dụng để dự đoán output y, mô hình này phù hợp để dự đoán một giá trị thực của đầu ra không bị chặn trên và chặn dưới. Còn trong **PLA**, đầu ra chỉ nhận một trong hai giá trị 1 hoặc −1, phù hợp với các bài toán **Binary Classification**. 

>> Tuy vậy, mình lại không sử dụng **Linear Regression**, hay thậm chí là **PLA**, mặc dù **PLA** rất phù hợp với bài toán của mình. Mà mình lựa chọn **Logistic Regression**. Mô hình này giống với **Linear Regression** ở khía cạnh đầu ra là số thực, và giống với **PLA** ở việc đầu ra bị chặn (trong đoạn 0 -> 1). Mặc dù trong tên có chứa từ "**regression**", tuy vậy **Logistic Regression** thường được sử dụng nhiều cho các bài toán **classification**. Đánh giá mô hình mình thấy nó rất là linh hoạt va dễ sử dụng. Do vậy, mình lựa chọn **Logistic Regression** cho bài toán này.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
model = LogisticRegression(C=1, random_state=0)

vectorize_model_pipeline = Pipeline([
    ('count_vectorizer', count_vectorizer),
    ('model', model)
])

**Mặc dù đã xử lý dữ liệu ở phần phía trên, tuy vậy, dữ liệu đầu vào của mình vẫn là dạng Text (nó sạch hơn một chút thôi).
Để sử dụng dữ liệu văn bản cho mô hình dự đoán, văn bản phải được phân tích cú pháp để loại bỏ một số từ nhất định - quá trình này được gọi là mã hóa . Sau đó, những từ này cần được mã hóa dưới dạng số nguyên hoặc giá trị dấu phẩy động, để sử dụng làm đầu vào trong thuật toán học máy. Quá trình này được gọi là trích xuất đặc trưng (hoặc vectơ hóa) .**

> Scikit-learning CountVectorizerđược sử dụng để chuyển đổi một bộ sưu tập các tài liệu văn bản thành một vectơ có số lượng thuật ngữ / mã thông báo. Nó cũng cho phép xử lý trước dữ liệu văn bản trước khi tạo biểu diễn vectơ. Chức năng này làm cho nó trở thành một mô-đun biểu diễn tính năng rất linh hoạt cho văn bản.

**Hay nói tóm gọn lại là mình sử dụng CountVectorizer để chuyển dữ liệu đầu vào từ Text sang Vectơ****

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train['question_text_cleaned'], df_train['target'], test_size = 0.3)
vectorize_model_pipeline.fit(X_train, y_train)

In [ ]:
predictions = vectorize_model_pipeline.predict(X_test)

In [ ]:
print('Accuracy :', accuracy_score(y_test, predictions))
print('F1 score :', accuracy_score(y_test, predictions))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

In [ ]:
# Cleaning the questions
df_test['question_text_cleaned'] = df_test.question_text.apply(lambda x: clean(x, True))

In [ ]:
df_test['prediction'] = vectorize_model_pipeline.predict(df_test['question_text_cleaned'])

In [ ]:
df_final = df_test[['qid','prediction']]
df_final.set_index('qid', inplace = True)

In [ ]:
df_final.head(5)

In [ ]:
df_final.to_csv('submission.csv')

# 3. Đánh giá mô hình Logistic Regression
**Logistic Regression là mô hình dễ sử dụng, và dễ hiểu, số điểm nó đạt được cũng tương đối cao:**

Private Score: **0.51781**

Public Score:  **0.51129**

**Logistic Regression thực ra được sử dụng nhiều trong các bài toán Classification.**

Mặc dù có tên là **Regression**, tức một mô hình cho fitting, **Logistic Regression** lại được sử dụng nhiều trong các bài toán **Classification**. Sau khi tìm được mô hình, việc xác định class y cho một điểm dữ liệu x được xác định bằng việc so sánh hai biểu thức xác suất:
        **P(y = 1|x; w); P(y = 0|x; w)**
Nếu biểu thức thứ nhất lớn hơn thì ta kết luận điểm dữ liệu thuộc class 1, ngược lại thì nó thuộc class 0. Vì tổng hai biểu thức này luôn bằng 1 nên một cách gọn hơn, ta chỉ cần xác định xem **P(y = 1|x; w)** lớn hơn 0.5 hay không. Nếu có, class 1. Nếu không, class 0.

***Logistic Regression đạt được số điểm khá là khả quan, tuy vậy, nó vẫn không thực sự là một điểm cao, vì vậy, mình xử dụng một mô hình khác đối với bài toán này, mong rằng số điểm nó sẽ cao hơn***

# 4. Training dữ liệu với mô hình khác